# Driver Kaggle

I start by downloading and and unziping the competition data from Kaggle and ensure my directory structure looks like this
```
utils/
    vgg16.py
    utils.py
lesson1/
    driver_kaggle.ipynb
    data/
        driver/
            train/
                ...
            test/
                ...
```
```$ sudo apt-get install python-pip python-dev build-essential 
$ sudo pip install --upgrade pip 
$ sudo pip install --upgrade virtualenv 
$ sudo apt-get install kaggle-cli
$ kg download -u 'username' -p 'pwd' -c 'state-farm-distracted-driver-detection'
$ sudo apt-get install ffmpeg #if need to get out of MemoryError```

You can download the data files from the competition page [here](https://www.kaggle.com/c/state-farm-distracted-driver-detection/data) or you can download them from the command line using the [Kaggle CLI](https://github.com/floydwch/kaggle-cli).

Then I launch your notebook inside the lesson1 directory
```
cd lesson1
jupyter notebook
```

In [1]:
%cd
%cd /Users/ssicard/goinfre/nbs/lesson1
%pwd

/Users/ssicard
/Volumes/Storage/goinfre/ssicard/nbs/lesson1


'/Volumes/Storage/goinfre/ssicard/nbs/lesson1'

In [2]:
%cd /Users/ssicard/goinfre/nbs/lesson1/data/driver/train

/Volumes/Storage/goinfre/ssicard/nbs/lesson1/data/driver/train


In [3]:
%mkdir ../valid
%mkdir ../results

## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories 
3. Finetune and Train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

In [13]:
%matplotlib inline
#path = '/Users/ssicard/goinfre/nbs/lesson1/data/driver/'
result_path = '/Users/ssicard/goinfre/nbs/lesson1/data/driver/results/'
%cd
%cd /Users/ssicard/goinfre/nbs/

/Users/ssicard
/Volumes/Storage/goinfre/ssicard/nbs


In [14]:
import os, json
from glob import glob
import numpy as np

from utils import *
%cd
%cd /Users/ssicard/goinfre/nbs/lesson1/data/driver/train

/Users/ssicard
/Volumes/Storage/goinfre/ssicard/nbs/lesson1/data/driver/train


#### <font color=blue> Create validation set and sample</font>

In [15]:
%mkdir ../valid

In [16]:
for d in glob('c?'):
    os.mkdir('../valid/'+d)
    g = glob(d+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(5):
        os.rename(shuf[i], '../valid/' + shuf[i])

In [17]:
%mkdir ../sample

In [18]:
for d in glob('c?'):
    os.mkdir('../sample/'+d)
    g = glob(d+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(50):
        os.rename(shuf[i], '../sample/' + shuf[i])

In [19]:
%cd /Volumes/Storage/goinfre/ssicard/nbs

/Volumes/Storage/goinfre/ssicard/nbs


In [1]:
%env PATH=$PATH:/usr/local/cuda/bin

env: PATH=$PATH:/usr/local/cuda/bin


In [2]:
%cd /Users/ssicard/goinfre/nbs/lesson1/data/driver/

/Volumes/Storage/goinfre/ssicard/nbs/lesson1/data/driver


In [3]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.configdefaults): Only clang++ is supported. With g++, we end up with strange g++/OSX bugs.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.configdefaults): We are not able to detect the number of CPU cores. We disable openmp by default. To remove this warning, set the environment variable OMP_NUM_THREADS to the number of threads you want theano to use.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu0 is not av

In [19]:
%matplotlib inline
from __future__ import print_function, division
path = "/Volumes/Storage/goinfre/ssicard/nbs/lesson1/data/driver/"
#path = "/Volumes/Storage/goinfre/ssicard/nbs/lesson1/data/driver/sample/"
from imp import reload
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

In [8]:
import sys
sys.path

['',
 '/Library/Frameworks/Python.framework/Versions/3.4/lib/python34.zip',
 '/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4',
 '/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/plat-darwin',
 '/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/lib-dynload',
 '/Users/ssicard/Library/Python/3.4/lib/python/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages',
 '/Users/ssicard/Library/Python/3.4/lib/python/site-packages/IPython/extensions',
 '/Users/ssicard/.ipython']

In [20]:
batch_size=2

In [21]:
batches = get_batches(path+'sample', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 500 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [22]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 21124 images belonging to 10 classes.
Found 50 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


In [23]:
trn = get_data(path+'sample')
val = get_data(path+'valid')

Found 500 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [24]:
save_array(path+'results/val.dat', val)
save_array(path+'results/trn.dat', trn)

In [25]:
val = load_array(path+'results/val.dat')
trn = load_array(path+'results/trn.dat')

In [28]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.samples, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.samples)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.samples, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.samples)
    return model

In [ ]:
model = conv1(batches)

/Users/ssicard/Library/Python/3.4/lib/python/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/Users/ssicard/Library/Python/3.4/lib/python/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys
/Users/ssicard/Library/Python/3.4/lib/python/site-packages/ipykernel_launcher.py:18: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/ssicard/Library/Python/3.4/lib/python/site-packages/ipykernel_launcher.py:18: Us

Epoch 1/2
214/500 [===========>..................] - ETA: 265s - loss: 2.5638 - acc: 0.1332